In [1]:
import pandas as pd
from templates import *
from builder import *

In [2]:
job_file = './data/experience.csv'
skill_file = './data/skills.csv'
basic_info = './data/basic_info.csv'

In [3]:
fname = './data/data.yaml'
test = yaml_builder(fname)
template = template_basic()
keys = ['programming']
output = 'test.pdf'
test.build_resume(template, keys = keys, output = output)

{'Programming Skills': ['Python', 'Javascript', 'MATLAB', 'Powershell', 'C++'], 'Technical Skills': ['Machine Learning', 'NLP', 'Pattern Recognition', 'Digital Forensics', 'Web Development']}
{'work': {'Health Partner Plans': {'location': 'Philadelphia, PA', 'title': 'Privacy and Security Intern', 'date': 'April 2019 - December 2019', 'detail': ['Managed data access through company governance software', 'Automated daily computer activity reports with Powershell to improve efficiency', 'Collaborated with IT Staff to promote cybersecurity and role governance within company']}}, 'projects': {'Interview Bot': {'date': 'July 2022 - present', 'detail': ['Developed an interactive FAQ bot to respond to commonly asked interview questions', 'Built a database that consists of over 100 interview questions', 'Created using sci-kit learn and spaCY (natural language processing toolkit)']}, 'Blog / rayyungdev.github.io': {'date': 'July 2022 - present', 'detail': ['Designed and created a personal site 

In [4]:
test.build_experiences(keys)

{'work': {'Health Partner Plans': {'location': 'Philadelphia, PA',
   'title': 'Privacy and Security Intern',
   'date': 'April 2019 - December 2019',
   'detail': ['Managed data access through company governance software',
    'Automated daily computer activity reports with Powershell to improve efficiency',
    'Collaborated with IT Staff to promote cybersecurity and role governance within company']}},
 'projects': {'Interview Bot': {'date': 'July 2022 - present',
   'detail': ['Developed an interactive FAQ bot to respond to commonly asked interview questions',
    'Built a database that consists of over 100 interview questions',
    'Created using sci-kit learn and spaCY (natural language processing toolkit)']},
  'Blog / rayyungdev.github.io': {'date': 'July 2022 - present',
   'detail': ['Designed and created a personal site from scratch to as a means to learn React and showcase personal projects',
    'Built using Gatsby as a static site generator that is now hosted on GitHub']},

In [5]:
template = template_basic()
resume = resume_builder(job_file, skill_file, basic_info)

In [6]:
resume.jobs

,type,tags,skills,start,end,title,company,location,detail
5,P,"[machine learning, software development, pytho...","Python, scikit-learn, Natural Language Processing",2022-07-01,NaT,Interview Bot,NaN,NaN,[Developed an interactive FAQ bot to respond t...
6,P,"[programming, web development, technology, jav...","React, Gatsby, Web development",2022-07-01,NaT,Blog / rayyungdev.github.io,NaN,NaN,[Designed and created a personal site from scr...
7,P,"[software development, programming, python, pa...","Python, Object Oriented Programming, Database",2022-07-01,2022-08-01,Resume Builder,NaN,NaN,[Developed a resume building application that ...
0,P,"[machine learning, software development, data ...","Pytorch, Deep Learning, AWS",2022-03-01,2022-06-01,DialogRPT / NLP with Deep Learning,NaN,NaN,[Built deep learning models to predict engagin...
2,P,"[machine learning, software development, data ...","Python, Machine Learning",2022-03-01,2022-06-01,Predicting Social Vulnerability,NaN,NaN,[Used binary classification techniques to pred...
3,P,"[machine learning, software development, matla...","MATLAB, Image Processing",2020-09-01,2021-04-01,Cell Segmentation and Tracking Algorithm,NaN,NaN,[Optimized thresholds for phase contrast algor...
1,J,"[IT, security, programming, technology]",NaN,2019-04-01,2020-12-01,Privacy and Security Intern,Health Partner Plans,"Philadelphia, PA",[Managed data access through company governanc...
4,J,"[IT, security, technology]","Active Directory, Data Management",2018-03-01,2020-09-01,Information Technology,Drexel University,"Philadelphia, PA",[Assisted users with personable expertise to t...
